# 1.Layers and Moudules

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2194, -0.0418, -0.0379,  0.1870, -0.2197,  0.1605, -0.1630, -0.0650,
         -0.0211,  0.0874],
        [ 0.1696, -0.1325, -0.0175,  0.0846, -0.1807,  0.1727, -0.1142, -0.0143,
          0.0031, -0.0019]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.hidden(x))
        x = self.out(x)
        return x

In [5]:
net = MLP()
net(X)

tensor([[-0.0039,  0.0152,  0.1388,  0.1025, -0.0516,  0.1123, -0.0067,  0.1882,
          0.0096, -0.0343],
        [-0.0290, -0.0484,  0.0491,  0.0536, -0.0659, -0.0258,  0.0252,  0.1446,
         -0.0478, -0.0127]], grad_fn=<AddmmBackward0>)

In [10]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for i, layer in enumerate(args):
            self._modules[str(i)] = layer

    def forward(self, x):
        for layer in self._modules.values():
            x = layer(x)
        return x

In [11]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0281,  0.0556, -0.1922,  0.0904, -0.2881, -0.1891, -0.1664, -0.0068,
         -0.2526, -0.4884],
        [-0.0191,  0.0815, -0.1905,  0.0982, -0.2704, -0.0981, -0.0501, -0.0482,
         -0.2328, -0.3023]], grad_fn=<AddmmBackward0>)

In [17]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = F.relu(self.linear(x))
        x = x @ self.rand_weight + 1
        x = self.linear(x)
        while x.abs().sum() > 1:
            x = x / 2
        return x.sum()

In [19]:
net = FixedHiddenMLP()
net(X)

tensor(-0.5768, grad_fn=<SumBackward0>)

In [20]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0231, grad_fn=<SumBackward0>)

# 2.Parameters

In [21]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.2004],
        [-0.2505]], grad_fn=<AddmmBackward0>)

In [23]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1624, -0.1042, -0.2168,  0.2737, -0.0596, -0.2368,  0.1440, -0.2769]])), ('bias', tensor([-0.2962]))])


In [26]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].bias.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2962], requires_grad=True)
tensor([-0.2962])
None


In [28]:
print(net.state_dict())

OrderedDict([('0.weight', tensor([[-0.4155,  0.1246,  0.3990,  0.1245],
        [-0.1310, -0.0295,  0.1886,  0.3743],
        [-0.1539, -0.1670, -0.3281, -0.4133],
        [-0.0277,  0.4263,  0.1148,  0.4047],
        [ 0.2433, -0.2173,  0.0650,  0.3640],
        [ 0.2087, -0.3929, -0.4194, -0.0789],
        [-0.4113,  0.0838, -0.4516, -0.2986],
        [ 0.1564,  0.0105, -0.1543,  0.1535]])), ('0.bias', tensor([ 0.0510,  0.3831,  0.3370,  0.2289, -0.1745, -0.2467,  0.4449,  0.0989])), ('2.weight', tensor([[-0.1624, -0.1042, -0.2168,  0.2737, -0.0596, -0.2368,  0.1440, -0.2769]])), ('2.bias', tensor([-0.2962]))])


In [31]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [32]:
net.state_dict()['2.bias'].data

tensor([-0.2962])

In [33]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3097],
        [0.3097]], grad_fn=<AddmmBackward0>)

In [35]:
print(net)
print(rgnet)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)
Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [44]:
rgnet[0][1][0].bias.data == rgnet.state_dict()['0.block 1.0.bias']

tensor([True, True, True, True, True, True, True, True])

In [46]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data, net[0].bias.data

(tensor([[-0.0129,  0.0097,  0.0009, -0.0104],
         [ 0.0057, -0.0007,  0.0116, -0.0134],
         [-0.0109,  0.0004, -0.0020,  0.0028],
         [-0.0194,  0.0108, -0.0061, -0.0119],
         [-0.0015, -0.0029, -0.0028, -0.0066],
         [-0.0007,  0.0111, -0.0061,  0.0060],
         [-0.0169,  0.0028,  0.0111, -0.0098],
         [ 0.0015,  0.0207,  0.0088,  0.0074]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [47]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
net.apply(init_constant)
net[0].weight.data, net[0].bias.data

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [48]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(init_xavier)
net[2].apply(init_42)
net[0].weight.data, net[2].weight.data

(tensor([[ 0.1959, -0.4163,  0.1411, -0.5990],
         [-0.3384, -0.5987,  0.5731, -0.2520],
         [ 0.1131, -0.0458, -0.0198,  0.3329],
         [-0.6309,  0.2854,  0.0064,  0.6873],
         [ 0.2169,  0.3474,  0.4490, -0.6970],
         [ 0.2974, -0.2639, -0.6018,  0.3482],
         [-0.4798, -0.3889,  0.5052,  0.0280],
         [ 0.3975, -0.0020,  0.6354,  0.3072]]),
 tensor([[42., 42., 42., 42., 42., 42., 42., 42.]]))

In [49]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.9243, -0.0000,  0.0000, -6.7086],
        [ 7.6318, -8.7489, -7.5500, -0.0000]], grad_fn=<SliceBackward0>)

In [50]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [52]:
net[2].weight.data[0]

tensor([ 1.0000e+02,  2.6811e-01, -2.8872e-01,  3.0996e-01,  2.3871e-01,
         7.6719e-02,  7.2497e-02, -3.7749e-03])

# 3.Lazy initialization

In [53]:
import torch
from torch import nn
from d2l import torch as d2l

In [54]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

In [55]:
net[0].weight

<UninitializedParameter>

In [71]:
X = torch.rand(2, 20)
net[0].weight.shape

torch.Size([256, 20])

# 4.Custom Layer

In [89]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [90]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [91]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [92]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-1.8626e-09, grad_fn=<MeanBackward0>)

In [93]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [94]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.6338, -0.5496,  0.1445],
        [ 0.6229,  1.0646,  0.5794],
        [-0.6584, -0.2948,  0.8372],
        [-1.2387, -0.0234, -0.4122],
        [ 0.2116, -0.1846, -0.3381]], requires_grad=True)

In [95]:
linear(torch.rand(2, 5))

tensor([[0.0000, 1.2809, 0.2498],
        [0.0000, 1.6815, 0.0000]])

In [96]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.9986],
        [0.0000]])

# 5.File I/O

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [98]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

C:\Users\qszhe\AppData\Local\Temp\ipykernel_43312\2924237495.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x2, y2 = torch.load('x-files')


(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [99]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

C:\Users\qszhe\AppData\Local\Temp\ipykernel_43312\2269971588.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mydict2 = torch.load('mydict')


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [3]:
torch.save(net.state_dict(), 'mlp.params')

In [4]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

C:\Users\qszhe\AppData\Local\Temp\ipykernel_44516\2014828273.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clone.load_state_dict(torch.load('mlp.params'))


MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [5]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# 6.GPU

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [13]:
X = torch.ones(2, 3, device='cuda')
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [8]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [15]:
X + Z

tensor([[2., 2., 2.],
        [2., 2., 2.]], device='cuda:0')

In [16]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu()) # move net to GPU

In [17]:
net[0].weight.data.device

device(type='cuda', index=0)